In [ ]:
import os
import cv2
import time
import shutil
import random
import inspect
#import imageio as im
import numpy as np
import mahotas as mh
from PIL import Image
from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#from skimage import measure, filters

In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs
from skimage import measure

import skimage.filters as filters
import skimage.measure as measure
from sklearn.metrics import classification_report
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
######################################################################################################
experiment = "test3" 
######################################################################################################
real = f"{working_dir}/{experiment}/real_images"
#########################################################################################################

directories = {
    "normalized": f"{working_dir}/{experiment}/normalized_images",
    "cropped": f"{working_dir}/{experiment}/cropped_images",
    "npy": f"{working_dir}/{experiment}/pre_processing/npy",
    "originals": f"{working_dir}/{experiment}/pre_processing/originals",
    "masks": f"{working_dir}/{experiment}/pre_processing/masks",
    "test": f"{working_dir}/{experiment}/pre_processing/test",
    "s": f"{working_dir}/{experiment}/S",
    "crop_original": f"{working_dir}/{experiment}/pre_processing/crop_originals",
    "crop_masks": f"{working_dir}/{experiment}/pre_processing/crop_masks",
    "op": f"{working_dir}/{experiment}/post_processing/original_prediction",
    "pp": f"{working_dir}/{experiment}/post_processing/processed_prediction"
    
}

In [ ]:
def readpaths(directories):
    for key, path in directories.items():
        globals()[key] = path
        
def pros(binary_mask):
    import matplotlib.pyplot as plt
    from io import StringIO
    import sys
    # ...

    fig, axs = plt.subplots(1, 5, figsize=(15, 5))

    # Operation 1
    binary_mask = (binary_mask * 255).astype('uint8')
    labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    numbi = int(np.average(sizes))
    large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
    min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
    too_small = np.where(sizes < min_region_size)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
    axs[0].imshow(labeled)
    #axs[0].imshow(labeled_only_big)
    axs[0].set_title('Operation 1')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(labeled_only_big)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[0].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[0].transAxes)

    # Operation 2
    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1

    axs[1].imshow(binary_mask)
    axs[1].set_title('Operation 2')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[1].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[1].transAxes)

    # Operation 3
    binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)

    axs[2].imshow(binary_mask_closed)
    axs[2].set_title('Operation 3')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_closed)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[2].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[2].transAxes)

    # Operation 4
    binary_mask= binary_mask.reshape(binary_mask.shape[0],binary_mask.shape[1])
    binary_mask_open = mh.morph.close_holes(binary_mask)

    axs[3].imshow(binary_mask_open)
    axs[3].set_title('Operation 4')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_open)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[3].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[3].transAxes)

    # Operation 5
    for i in range(3):
        binary_mask_dialate = mh.morph.dilate(binary_mask_open)

    axs[4].imshow(binary_mask_dialate)
    axs[4].set_title('Operation 5')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_dialate)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[4].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[4].transAxes)

    plt.show()
    
def reshape(dir):
    for z in tqdm(sorted(os.listdir(dir))):
        if (z.endswith("tif")):
            img = mh.imread(os.path.join(dir, z))
            img.shape
            img = img.reshape(img.shape[0],img.shape[1],1)
    
def labels(binary_mask,z):
    print()
    print()
    print(z)
    # Calculate number_1
    labeled, num = mh.label(binary_mask)
    print("Number_1:", num)

    # Calculate region sizes
     # Calculate number_1
    labeled, num = mh.label(binary_mask)
    print("Number_1:", num)
    region_sizes = measure.regionprops(labeled, intensity_image=binary_mask)
    sizes = [region.area for region in region_sizes]

    # Find min_region_size
    sizes.sort()
    numbi = int(np.average(sizes))
    large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
    if large_region_start_index < len(sizes):
        min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
    else:
        min_region_size = sizes[-1]
    print("Min_region_size:", min_region_size)
    #print ('sizes:',sizes)
    a = len(sizes)
    print ('length of sizes =',a)

    
    # Calculate large_regions
    num_large_regions = 0
    for region in region_sizes:
        if region.area > min_region_size:
            num_large_regions += 1
    print("Large_regions:", num_large_regions)

    # Calculate threshold
    thresh = filters.threshold_otsu(binary_mask)
    #print(labels_name[jk])
    print("Threshold:", thresh)

In [ ]:
 !pip install pynvml

In [ ]:
def gpustat():
    !pip install pynvml
    import pynvml
    import time
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    for i in range(device_count):
            handle = pynvml.nvmlDeviceGetHandleByIndex(i)
            name = pynvml.nvmlDeviceGetName(handle)
            serial = pynvml.nvmlDeviceGetSerial(handle)
            util = pynvml.nvmlDeviceGetUtilizationRates(handle)
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            print(f"{name} - {serial} - Utilization: {util.gpu}% - Timestamp: {timestamp}")
    pynvml.nvmlShutdown()

In [ ]:
import pynvml
import time
pynvml.nvmlInit()
device_count = pynvml.nvmlDeviceGetCount()
for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        name = pynvml.nvmlDeviceGetName(handle)
        serial = pynvml.nvmlDeviceGetSerial(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(f"{name} - {serial} - Utilization: {util.gpu}% - Timestamp: {timestamp}")
pynvml.nvmlShutdown()

In [ ]:
!nvidia-smi

In [ ]:
print ('hello')

In [ ]:
gpustat()

In [ ]:
readpaths(directories)

In [ ]:
import os
import random
import cv2
import mahotas as mh


file_list = sorted(os.listdir(op))
z = random.choice([f for f in file_list if f.endswith("tif")])
img = mh.imread(os.path.join(op, z))
print(z)
plt.imshow(img)
#         image = cv2.imread(os.path.join(cropped, f))
#         print (f)

# # # Get a list of all image files in the directory
# # image_files = [f for f in os.listdir(path) if f.endswith('.tif')]

# # # Randomly select an image file
# # selected_image = random.choice(image_files)

# # # Load the selected image
# # image = cv2.imread(os.path.join(path, selected_image))

# # Show the selected image
# cv2.imshow('Selected Image', image)


In [ ]:
img.shape

In [ ]:
binary_mask = img

In [ ]:
#binary_mask = np.array(binary_mask)

In [ ]:
labeled, number = mh.label(binary_mask)
sizes = mh.labeled.labeled_size(labeled)

In [ ]:
a =sizes.max()
b =np.average(sizes)

In [ ]:
c = a-b
print(a)
print(b)
print(c)

In [ ]:
plt.imshow(binary_mask, cmap='gray')
plt.show()

In [ ]:
def findings(a):
    for i in range(a):
        import os
        import random
        import cv2
        import mahotas as mh
        import skimage.morphology as skm
        import numpy as np


        file_list = sorted(os.listdir(op))
        z = random.choice([f for f in file_list if f.endswith("tif")])
        img = mh.imread(os.path.join(op, z))
        print(z)
        #fig.set_size_inches(30, 30)
        plt.imshow(img)
        image = (img * 255).astype('uint8')
        labels(image,z)
        ########################################
        dst = cv2.fastNlMeansDenoising(img, None,60, 40, 40)
        #(SOURCE, NEW IMAGE, FILTER STRENGTH MAX -, SIZE OF WINDOW FOR PIXEL SEARCH, TO AVERAGE PIXELS)
        # Display the original image and the denoised image
        plt.subplot(121), plt.imshow(img)
        plt.subplot(122), plt.imshow(dst)
        plt.show()
        ###################################################
        # Check if image is grayscale
        if len(img.shape) == 2:
            gray = img
        else:
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply binary threshold with a threshold value of 150
        _, binary_mask = cv2.threshold(gray, 100
                                  , 255, cv2.THRESH_BINARY)

        # Display the original image and the thresholded image side by side
        fig, axs = plt.subplots(1, 2)
        axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[0].set_title('Original Image')
        axs[1].imshow(binary_mask, cmap='gray')
        axs[1].set_title('Thresholded Image')
        plt.show()
        ########################################
        newops(binary_mask,z)
        #pros(binary_mask)
        

In [ ]:
def newops(binary_mask,z):
    import matplotlib.pyplot as plt
    from io import StringIO
    import sys
    import scipy
    from scipy import ndimage
    import skimage.morphology as skm
    # ...

    fig, axs = plt.subplots(1, 5, figsize=(15, 5))
    print ('function -newops',z)
    binary_mask = binary_mask
    # Operation 1
    binary_mask = (binary_mask * 255).astype('uint8')
    labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    numbi = int(np.average(sizes))
    large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
    min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
    too_small = np.where(sizes < min_region_size)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
    axs[0].imshow(labeled)
    #axs[0].imshow(labeled_only_big)
    axs[0].set_title('Operation 1')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(labeled_only_big,z)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[0].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[0].transAxes)

    # Operation 2
    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1

    axs[1].imshow(binary_mask)
    axs[1].set_title('Operation 2')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask,z)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[1].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[1].transAxes)

    # Operation 3
    binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)

    axs[2].imshow(binary_mask_closed)
    axs[2].set_title('Operation 3')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_closed,z)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[2].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[2].transAxes)

    # Operation 4
    binary_mask= binary_mask.reshape(binary_mask.shape[0],binary_mask.shape[1])
    binary_mask_open = mh.morph.close_holes(binary_mask)

    axs[3].imshow(binary_mask_open)
    axs[3].set_title('Operation 4')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_open,z)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[3].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[3].transAxes)

    # Operation 5
    for i in range(3):
        binary_mask_dialate = mh.morph.dilate(binary_mask_open)

    axs[4].imshow(binary_mask_dialate)
    axs[4].set_title('Operation 5')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_dialate,z)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[4].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[4].transAxes)

    plt.show()
    ####opertion -6
    min_pixel = 1
    uniq_l, uniq_num = mh.label(binary_mask_dialate)
    uniq_sizes = mh.labeled.labeled_size(uniq_l)
    range_num=int(np.average(uniq_sizes))
    print (range_num)
    for i in range(range_num):
        labeled, op6_num = mh.label(binary_mask_dialate)
        region_sizes = measure.regionprops(labeled, intensity_image=binary_mask_dialate)
        sizes = [region.area for region in region_sizes]
        sizes.sort()
        numbi = int(np.average(sizes))
        large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
        if large_region_start_index < len(sizes):
            min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
        else:
            min_region_size = sizes[-1]
        op6_largeregions = 0
        for region in region_sizes:
            if region.area > min_region_size:
                op6_largeregions += 1 
                
        if op6_largeregions >=20:
            binary_mask_dialate, number = mh.label(binary_mask_dialate)
            sizesa = mh.labeled.labeled_size(binary_mask_dialate)
            small = np.where(sizesa < min_pixel)
            binary_mask_dialate = mh.labeled.remove_regions(binary_mask_dialate, small)
            latest_binary_mask = binary_mask_dialate
            #print ('failed operation-6')
            #plt.imshow(latest_binary_mask)
            #labels(latest_binary_mask)
            #plt.show()
            
        elif op6_num == (op6_largeregions+1):
            print ('success operation-6')
            #binary_mask_dialate[binary_mask_dialate > 0] = 1
            #plt.imshow(binary_mask_dialate)
            labels(binary_mask_dialate,z)
            #plt.show()
            ###################
            # Define a structuring element
            struct = ndimage.generate_binary_structure(2, 2)
            # Apply closing to the binary image
            closed_mask = ndimage.binary_closing(binary_mask_dialate, structure=struct, iterations=2)
            
            plt.imshow(closed_mask)
            plt.show()
            save_path = os.path.join(pp, z)
            print("Saving image to:", save_path)
            mh.imsave(os.path.join(pp,z),closed_mask)

            break
        else:
            binary_mask_dialate, number = mh.label(binary_mask_dialate)
            sizesb = mh.labeled.labeled_size(binary_mask_dialate)            
            #print ('failed - 2 operation-6')
            smallb = np.where(sizesb < min_pixel)
            binary_mask_dialate = mh.labeled.remove_regions(binary_mask_dialate, smallb)
            latest_binary_mask = binary_mask_dialate
            #plt.imshow(latest_binary_mask)
            #labels(latest_binary_mask)
            #plt.show()
        min_pixel += 1
        #print ('#####################',min_pixel,'#####################')
        binary_mask_dialate = latest_binary_mask
        #print(sizes)
        #print(sizesa)
        
        #pros(binary_mask_dialate)

In [ ]:
#newops(binary_mask)

In [ ]:
findings(1812)

In [ ]:
def checkings(a):
    for i in range(a):
        import os
        import random
        import cv2
        import mahotas as mh
        import numpy as np
        import matplotlib.pyplot as plt
        from io import StringIO
        import sys
        file_list = sorted(os.listdir(op))
        z = random.choice([f for f in file_list if f.endswith("tif")])
        img = mh.imread(os.path.join(op, z))
        print(z)
        #fig.set_size_inches(30, 30)
        plt.imshow(img)
        print
        binary_mask = (img * 255).astype('uint8')
        labels(binary_mask)
        
        
        # .... .. . . . .. . . . . ............................................#
        fig, axs = plt.subplots(1, 5, figsize=(15, 5))

        # Operation 1
        binary_mask = (binary_mask * 255).astype('uint8')
        labeled, number = mh.label(binary_mask)
        sizes = mh.labeled.labeled_size(labeled)
        numbi = int(np.average(sizes))
        large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
        min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
        too_small = np.where(sizes < min_region_size)
        labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
        axs[0].imshow(labeled)
        #axs[0].imshow(labeled_only_big)
        axs[0].set_title('Operation 1')

        # Capture the output of the labels function
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()

        labels(labeled_only_big)

        # Restore stdout
        sys.stdout = old_stdout

        # Display the captured output as text below the subplot
        axs[0].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[0].transAxes)

        # Operation 2
        binary_mask = labeled_only_big.copy()
        binary_mask[binary_mask > 0] = 1

        axs[1].imshow(binary_mask)
        axs[1].set_title('Operation 2')

        # Capture the output of the labels function
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()

        labels(binary_mask)

        # Restore stdout
        sys.stdout = old_stdout

        # Display the captured output as text below the subplot
        axs[1].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[1].transAxes)

        # Operation 3
        binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)

        axs[2].imshow(binary_mask_closed)
        axs[2].set_title('Operation 3')

        # Capture the output of the labels function
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()

        labels(binary_mask_closed)

        # Restore stdout
        sys.stdout = old_stdout

        # Display the captured output as text below the subplot
        axs[2].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[2].transAxes)

        # Operation 4
        binary_mask= binary_mask.reshape(binary_mask.shape[0],binary_mask.shape[1])
        binary_mask_open = mh.morph.close_holes(binary_mask)

        axs[3].imshow(binary_mask_open)
        axs[3].set_title('Operation 4')

        # Capture the output of the labels function
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()

        labels(binary_mask_open)

        # Restore stdout
        sys.stdout = old_stdout

        # Display the captured output as text below the subplot
        axs[3].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[3].transAxes)

        # Operation 5
#         for i in range(3):
#                 binary_mask_dialate = mh.morph.dilate(binary_mask_open)
        for ga in range(150, 0, -1):
            # Check if image is grayscale
            if len(img.shape) == 2:
                gray = img
            else:
                # Convert to grayscale
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Apply binary threshold with a threshold value of 150
            _, binary_mask = cv2.threshold(gray, ga, 255, cv2.THRESH_BINARY)
            labeled11, num4 = mh.label(binary_mask)
            region_sizes1 = measure.regionprops(labeled11, intensity_image=binary_mask)
            num_large_regions2 = 0
            sizes2 = [region.area for region in region_sizes1]
            # Find min_region_size
            sizes2.sort()
            numbi = int(np.average(sizes2))
            large_region_start_index1 = next((i for i in range(1, len(sizes2)) if sizes2[i] - sizes2[i-1] < numbi), len(sizes2))
            if large_region_start_index1 < len(sizes2):
                min_region_size2 = (sizes2[large_region_start_index1-1] + sizes2[large_region_start_index1]) // 2
            else:
                min_region_size2 = sizes2[-1]

            for region in region_sizes1:
                if region.area > min_region_size2:
                    num_large_regions2 += 1
            if num4 >= 20 and num_large_regions2 >= 20:
                break
        
        
        thickness=5
        # Define the kernel size for morphological operations
        kernel_size = 8
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
        # apply dilation to the binary mask
        dilated_mask = cv2.dilate(binary_mask, kernel)
        # find contours in the dilated mask
        contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # draw contours on the original image
        
        
        binary_mask23 = cv2.drawContours(binary_mask, contours, -1, (0, 0, 255), thickness)
        labeled3, number = mh.label(binary_mask23)
        binary_mask33 = labeled3.copy()
        binary_mask33[binary_mask33> 0] = 1
        axs[4].imshow(binary_mask33)
        axs[4].set_title('Operation 5')
        # Capture the output of the labels function
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()

        labels(binary_mask33)

        # Restore stdout
        sys.stdout = old_stdout

        # Display the captured output as text below the subplot
        axs[4].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[4].transAxes)

        plt.show()
        
        print('operation -6')
        print (min_region_size, min_region_size2)
        

In [ ]:
checkings(25)

In [ ]:
checkings(15)

In [ ]:
findings(15)

In [ ]:
def findings(a):
    for i in range(a):
        import os
        import random
        import cv2
        import mahotas as mh

        import numpy as np


        file_list = sorted(os.listdir(op))
        z = random.choice([f for f in file_list if f.endswith("tif")])
        img = mh.imread(os.path.join(op, z))
        print(z)
        #fig.set_size_inches(30, 30)
        plt.imshow(img)
        image = (img * 255).astype('uint8')
        labels(image,z)
        ########################################
        dst = cv2.fastNlMeansDenoising(img, None,60, 40, 40)
        #(SOURCE, NEW IMAGE, FILTER STRENGTH MAX -, SIZE OF WINDOW FOR PIXEL SEARCH, TO AVERAGE PIXELS)
        # Display the original image and the denoised image
        plt.subplot(121), plt.imshow(img)
        plt.subplot(122), plt.imshow(dst)
        plt.show()
        ###################################################
        # Check if image is grayscale
        if len(img.shape) == 2:
            gray = img
        else:
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply binary threshold with a threshold value of 150
        _, binary_mask = cv2.threshold(gray, 100
                                  , 255, cv2.THRESH_BINARY)

        # Display the original image and the thresholded image side by side
        fig, axs = plt.subplots(1, 2)
        axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[0].set_title('Original Image')
        axs[1].imshow(binary_mask, cmap='gray')
        axs[1].set_title('Thresholded Image')
        plt.show()
        ########################################
        newops(binary_mask,z)
        #pros(binary_mask)
        

In [ ]:
findings(30)

In [ ]:
findings(20)

In [ ]:
findings(20)

In [ ]:
pros(binary_mask)

In [ ]:
binary_mask.shape

In [ ]:
import mahotas
print(mahotas.__version__)


In [ ]:
# import mahotas as mh

# # get labeled blobs
# labeled, num_blobs = mh.label(binary_mask)
# sizes = mh.labeled.labeled_size(labeled)
# # print sizes
# print("Sizes for image", labeled)
# for j in range(1, num_blobs):
#     print("Blob", j, "size:", sizes[j])

In [ ]:
binary_mask = (binary_mask * 255).astype('uint8')

In [ ]:
labeled, number = mh.label(binary_mask)
sizes = mh.labeled.labeled_size(labeled)

In [ ]:
sizes.max()

In [ ]:
sizes.min()

In [ ]:
np.average(sizes)

In [ ]:
####################

In [ ]:

alpha = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test2/pre_processing/originals'
beta = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test2/pre_processing/masks'

In [ ]:
z = '000000_000000_000292_000000_0_1.tif'
for z in tqdm(sorted(os.listdir(beta))):
    if (z.endswith("tif")):
        img = mh.imread(os.path.join(beta, z))
img.shape


In [ ]:
import numpy as np

default_z = '000000_000000_000292_000000_0_1.tif'
for file_name in tqdm(sorted(os.listdir(op))):
    if file_name == default_z:
        img = mh.imread(os.path.join(op, file_name))
        img = img.reshape(img.shape[0], img.shape[1], 1)
        img = (img * 255).astype(np.uint8)  # convert to integer data type
        mh.imsave(os.path.join(op, file_name), img)
        print(f"New shape of {file_name}: {img.shape}")

In [ ]:
print(a)

In [ ]:
reshape(op)

In [ ]:
img.shape

In [ ]:
reshape(op)

In [ ]:
import os
import random
import cv2
import mahotas as mh

import numpy as np


file_list = sorted(os.listdir(op))
z = random.choice([f for f in file_list if f.endswith("tif")])
img = mh.imread(os.path.join(op, z))
print(z)
fig.set_size_inches(30, 30)
plt.imshow(img)
image = (img * 255).astype('uint8')

In [ ]:
# Load the binary image
image = image

In [ ]:
img = cv2.imread(f'{op}/{z}',0)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Apply non-local means denoising
dst = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)

# Calculate the residual image
residual = img - dst
fig.set_size_inches(10, 10)
# Display the original image, the denoised image, and the residual image
plt.subplot(131), plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.subplot(132), plt.imshow(dst, cmap='gray')
plt.title('Denoised Image')
plt.subplot(133), plt.imshow(residual, cmap='gray')
plt.title('Residual Image')
plt.show()

In [ ]:
# Apply Gaussian adaptive thresholding
thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Display the original image and the thresholded image
titles = ['Original Image', 'Gaussian Adaptive Thresholding']
images = [img, thresh]

for i in range(2):
    plt.subplot(1, 2, i+1), plt.imshow(images[i], 'gray')
    plt.title(titles[i])
    plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
dst = cv2.fastNlMeansDenoising(img, None,60, 40, 40)
#(SOURCE, NEW IMAGE, FILTER STRENGTH MAX -, SIZE OF WINDOW FOR PIXEL SEARCH, TO AVERAGE PIXELS)
# Display the original image and the denoised image
plt.subplot(121), plt.imshow(img)
plt.subplot(122), plt.imshow(dst)
plt.show()

In [ ]:
# Check if image is grayscale
if len(img.shape) == 2:
    gray = img
else:
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply binary threshold with a threshold value of 150
_, binary_mask = cv2.threshold(gray, 100
                          , 255, cv2.THRESH_BINARY)

# Display the original image and the thresholded image side by side
fig, axs = plt.subplots(1, 2)
axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original Image')
axs[1].imshow(binary_mask, cmap='gray')
axs[1].set_title('Thresholded Image')
plt.show()


In [ ]:
labels(binary_mask)

In [ ]:
labeled, num = mh.label(binary_mask)
binary_maskg = labeled.copy()
binary_maskg[binary_maskg > 0] = 1

In [ ]:
sizes

In [ ]:
plt.imshow(binary_maskg)
plt.show()

In [ ]:
binary_mask_open = mh.morph.close_holes(binary_maskg)

In [ ]:
plt.imshow(binary_mask_open)
plt.show()

In [ ]:
for i in range(15):
    binary_mask1 = mh.morph.dilate(binary_mask_open)

In [ ]:
plt.imshow(binary_mask1)
plt.show()
a =type(binary_mask1)
print (a)

In [ ]:
import matplotlib.pyplot as plt

x = [1, 2, 3, 4, 5]
y = [1, 4, 9, 16, 25]

plt.plot(x, y)
plt.show()


In [ ]:
print(type(binary_mask1))

In [ ]:
labels(binary_mask)

In [ ]:
pros(binary_mask)

In [ ]:
# Check if image is grayscale
if len(img.shape) == 2:
    gray = img
else:
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply adaptive threshold with a block size of 11 and a constant value of 2
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 89, 34)

# Display the original image and the thresholded image side by side
fig, axs = plt.subplots(1, 2)
axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original Image')
axs[1].imshow(thresh, cmap='gray')
axs[1].set_title('Thresholded Image')
plt.show()

In [ ]:
# Invert the colors of the thresholded image
thresh_inv = cv2.bitwise_not(thresh)

# Display the inverted thresholded image
plt.imshow(thresh_inv, cmap='gray')
plt.title('Inverted Thresholded Image')
plt.show()

In [ ]:

labeled, number = mh.label(thresh_inv)
sizes = mh.labeled.labeled_size(labeled)

In [ ]:
sizes

In [ ]:
numbi = int(np.average(sizes))
    large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
    min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
    too_small = np.where(sizes < min_region_size)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

In [ ]:
a = float(sizes.max())
b = float(sizes.min())

In [ ]:
a = a -b

In [ ]:
too_small = np.where(sizes > a)
labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

In [ ]:
plt.imshow(labeled, cmap='gray')
plt.title('Inverted Thresholded Image')
plt.show()

In [ ]:
plt.imshow(labeled_only_big, cmap='gray')
plt.title('Inverted Thresholded Image')
plt.show()

In [ ]:
# Normalize image to 0-255
normalized = cv2.normalize(labeled, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)

# Apply Otsu's thresholding
_, thresh = cv2.threshold(normalized, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Perform morphological opening to remove small black regions
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Display the original image and the thresholded image side by side
fig, axs = plt.subplots(1, 2)
axs[0].imshow(opening, cmap='gray')
axs[0].set_title('Original Image')
axs[1].imshow(opening, cmap='gray')
axs[1].set_title('Thresholded Image')
plt.show()

In [ ]:
# Check if image is grayscale
if len(img.shape) == 2:
    gray = img
else:
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding with a block size of 11 and a constant of 2
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Show image
cv2.imshow('Thresholded Image', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Find the contours in the image
contours, _ = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Calculate the sizes and thicknesses of the contours
sizes = []
thicknesses = []
for contour in contours:
    size = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        thickness = 0
    else:
        thickness = 4 * np.pi * size / (perimeter ** 2)
    sizes.append(size)
    thicknesses.append(thickness)

# Calculate the mean and standard deviation of the sizes and thicknesses
mean_size = np.mean(sizes)
std_size = np.std(sizes)
mean_thickness = np.mean(thicknesses)
std_thickness = np.std(thicknesses)


In [ ]:
# Set the minimum size and thickness for the contours
min_size = mean_size - std_size
min_thickness = mean_thickness - std_thickness

# Calculate the distances between the contours
distances = []
for contour1 in contours:
    for contour2 in contours:
        if contour1 is not contour2:
            dist = cv2.pointPolygonTest(contour1, tuple(map(float, contour2[0][0])), True)
            distances.append(abs(dist))

# Calculate the median of the distances
median_distance = np.median(distances)

# Set the distance threshold for the pointPolygonTest function
distance_threshold = median_distance

# Create a mask to store the filtered contours
mask = np.zeros_like(image)

# Iterate through the contours
for contour in contours:
    # Calculate the size and thickness of the contour
    size = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        thickness = 0
    else:
        thickness = 4 * np.pi * size / (perimeter ** 2)

    # Check if the contour is close to any larger regions
    close_to_larger_region = False
    for other_contour in contours:
        if cv2.contourArea(other_contour) > size:
            dist = cv2.pointPolygonTest(contour1, tuple(map(float, contour2[0][0])), True)
            if abs(dist) < distance_threshold:
                close_to_larger_region = True
                break

    # If the contour is large enough or close to a larger region or thick enough, add it to the mask
    if size > min_size or close_to_larger_region or thickness > min_thickness:
        cv2.drawContours(mask, [contour], -1, 255, -1)


In [ ]:
# Apply the mask to the image to remove the small white dots
result = cv2.bitwise_and(image, mask)

# Show the result
cv2.imshow('Result', result)
cv2.waitKey(0)


In [ ]:

# Find the contours in the image
contours, _ = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Calculate the sizes and thicknesses of the contours
sizes = []
thicknesses = []
for contour in contours:
    size = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        thickness = 0
    else:
        thickness = 4 * np.pi * size / (perimeter ** 2)
    sizes.append(size)
    thicknesses.append(thickness)

# Calculate the mean and standard deviation of the sizes and thicknesses
mean_size = np.mean(sizes)
std_size = np.std(sizes)
mean_thickness = np.mean(thicknesses)
std_thickness = np.std(thicknesses)

# Set the minimum size and thickness for the contours
min_size = mean_size - std_size
min_thickness = mean_thickness - std_thickness

# Calculate the distances between the contours
distances = []
for contour1 in contours:
    for contour2 in contours:
        if contour1 is not contour2:
            dist = cv2.pointPolygonTest(contour1, tuple(map(float, contour2[0][0])), True)
            distances.append(abs(dist))

# Calculate the median of the distances
median_distance = np.median(distances)

# Set the distance threshold for the pointPolygonTest function
distance_threshold = median_distance

# Create a mask to store the filtered contours
mask = np.zeros_like(image)

# Iterate through the contours
for contour in contours:
    # Calculate the size and thickness of the contour
    size = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        thickness = 0
    else:
        thickness = 4 * np.pi * size / (perimeter ** 2)

    # Check if the contour is close to any larger regions
    close_to_larger_region = False
    for other_contour in contours:
        if cv2.contourArea(other_contour) > size:
            dist = cv2.pointPolygonTest(contour1, tuple(map(float, contour2[0][0])), True)
            if abs(dist) < distance_threshold:
                close_to_larger_region = True
                break

    # If the contour is large enough or close to a larger region or thick enough, add it to the mask
    if size > min_size or close_to_larger_region or thickness > min_thickness:
        cv2.drawContours(mask, [contour], -1, 255, -1)

# Apply the mask to the image to remove the small white dots
result = cv2.bitwise_and(image, mask)

# Show the result
cv2.imshow('Result', result)
cv2.waitKey(0)

In [ ]:
import os
os.environ['QT_QPA_PLATFORM'] = 'offscreen'